In [2]:
m=[1 2 3; 4 5 6; 5 6 1; 2 3 7]*10

4×3 Array{Int64,2}:
 10  20  30
 40  50  60
 50  60  10
 20  30  70

In [ ]:
function basic_travelcost(m)
    # cheapest cost from top left corner to every other square restricted to only South/East moves.
    # and correct FIRST direction to go in order to take that cheapest path
    Y=size(m,1)
    X=size(m,2)
    dir=ones(Int,(Y,X))
    cost=zeros(Int,(Y,X))
    for x=1:X,y=1:Y
        if x>1
            cost[y,x]=cost[y,x-1]+m[y,x-1]
            dir[y,x]  = y==1 ? 1 : dir[y,x-1]
        end
        if y>1
            t_cost=cost[y-1,x]+m[y-1,x]
            if t_cost<cost[y,x] || x==1
                cost[y,x]=t_cost
                dir[y,x] = x==1 ? 0 : dir[y-1,x]
            end
        end
    end
    dir[1,1]=9
    return cost, dir

end

In [ ]:
#helpers. Work with quartiles and shifts to generalize basic_travelcost
#to any move south/east/north/west (in that order 0,1,2,3) and any starting coordinate.
function replace!(x, KV::Pair{Int,Int}...)
    for (k,v) in KV
        x[x.==k] = v
    end
    x
end

#
q1(m) = flipdim(circshift(m, (-1,0)), 1)
q2(m) = q1(q3(m))
q3(m) = flipdim(circshift(m, (0,-1)), 2)
q4(m) = m

iq1(m,d) = q1(m), replace!(d, 0 => 2)
iq2(m,d) = q2(m), replace!(d, 0 => 2, 1 => 3)
iq3(m,d) = q3(m), replace!(d, 1 => 3)
iq4(m,d) = q4(m), d

shiftorigin(m, origin) = circshift(M, (1-origin[1], 1-origin[2]))
ishiftorigin(m, origin) = circshift(m, (origin[1]-1, origin[2]-1))

In [ ]:
#kommentar: om travelcost är mer än vad skeppet har mineraler så kan det inte åka dit.
#bör antagligen filtrera bord dom när skeppet ska välja vilken ruta som har bäst score.
function travelcost(M, origin)
    #Cheapest cost from some point to All Other points,
    #and what direction to go on First step to take that cheapest path.
    #south/east/north/west (in that order 0,1,2,3)
    m = shiftorigin(m, origin)
    expand(f) = x -> f(x...)
    c1,d1 = m |> q1 |> hej |> expand(iq1)
    c2,d2 = m |> q2 |> hej |> expand(iq2)
    c3,d3 = m |> q3 |> hej |> expand(iq3)
    c4,d4 = m |> q4 |> hej |> expand(iq4)
    
    D = cat(3, d1,d2,d3,d4)
    C = cat(3, c1,c2,c3,c4)
    v, i = findmin(C,3)
    
    cost = ishiftorigin(C[i][:,:,1], origin)
    first_direction = ishiftorigin(D[i][:,:,1], origin)
    return cost, first_direction
end

c, d = travelcost(m, [3,2])

In [ ]:
function score(M, ship, shipyard, threshold)
    #reward
    #minus
    #cost (from ship, to some point, and then to shipyard)
    mining = max.(0, M - threshold)
    leftovers = min.(mining, threshold)
    ignoreshipyard = M[shipyard...]
    cost1, direction1 = travelcost(0.1*M, ship)
    cost2, direction2 = travelcost(0.1*M, shipyard)
    
    cost = cost1 + cost2 + 0.1*leftovers - 0.1*ignoreshipyard
    direction = direction1
    
    score = mining - cost
    return score, direction
end